imports

In [ ]:
# imports
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# show first 5 rows of lorry_df
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes (customer_locations) via nearest neighbour sorting

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

states and constraints

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# function: finding next nearest customer node
def nearest_customer(currentState, customerList):
    #initialise dist dict
    dist = {}
    
    # loop through customerList
    for i in customerList:
        # check for all where customer != currentState
        if i != currentState:
            # update dist with available customer index and their relative weights
            dist.update({i:euclidean[i,currentState]})

    # initialise temp list
    temp = []

    # loop through keys of dist 
    for i in dist.keys():
        # add weights to temp
        temp.append(dist[i])
    
    # get lowest weight
    shortestDist = sorted(temp)[0]
    
    # find relative index of lowest weight
    nextCustomerIndex = np.where(euclidean[currentState]==shortestDist)
    
    # return next index with relative weight
    return int(nextCustomerIndex[0]), shortestDist

In [ ]:
# nearest_customer() test
nearest_customer(1, [2, 3, 1, 5])

In [ ]:
def nearest_depot(currentState, depotList):
    pass 